# Exploring an auto-encoder for FashionMNIST
An auto-encoder is a model that encodes some content (in this case an image), into a lower dimensional vector and then decodes it back to the original image. The encoder may be useful as a trained feature extractor. When evaluating the auto-encoder it is important to ensure the recreated content looks reasonable. This notebook shows an example of how one may visually inspect the recreated content from an auto-encoder.

This example uses ipywidgets, which may be enabled by running:
```
jupyter nbextension enable --py widgetsnbextension
```

## Import dependencies for this notebook

In [1]:
import urllib
import zipfile

import IPython.display as ipd
import numpy as np
import pandas as pd
import sidekick
from ipywidgets import interact_manual
from PIL import Image

## Download FashionMNIST
Download FashionMNIST to a tempfile. For details including license please see: [knowledge-center/fashion-mnist](https://peltarion.com/knowledge-center/documentation/datasets-view/datasets-used-in-tutorials/fashion-mnist-dataset)

In [2]:
fashion_mnist_path, _ = urllib.request.urlretrieve(
    'https://storage.googleapis.com/bucket-8732/fashion.zip')

## Extract content from the dataset into a DataFrame

In [3]:
# Open Peltarion platform compatible zipfile
with zipfile.ZipFile(fashion_mnist_path, 'r') as z:
    index_file = z.open('index.csv')
    dataset = pd.read_csv(index_file)
    dataset['image'] = dataset['image'].apply(
        lambda path: Image.open(z.open(path))
    )

## Connect to the auto-encoder
This auto-encoder takes both as in and output feature a greyscale image of shape 28x28 (i.e. matching the size of FashionMNIST images).

In [4]:
deployment = sidekick.Deployment(
    url='<url>',
    token='<token>',
    dtypes_in={'image': 'Image (28x28x1)'},
    dtypes_out={'image': 'Image (28x28x1)'}
)

## Create a generator which polls the deployment
For interactive exploration of data it us useful to use the predict_lazy method, which returns a generator that lazily polls the deployment when needed. Sidekick will make sure to batch requests to the deployment.

In [5]:
# Get predictions for all images in dataset
predictions = deployment.predict_lazy(dataset.to_dict('records'))

## Visualize a grid of output
Set the desired grid size and press `Run interact` to get a new grid of examples.

In [6]:
def show_next_grid(n_rows, n_columns):
    grid = Image.new('L', (n_columns * 28, n_rows * 28))
    for column in range(n_columns):
        for row in range(n_rows):
            grid.paste(next(predictions)['image'], (column * 28, row * 28))
    return grid

interact_manual(show_next_grid, n_rows=(1, 10), n_columns=(1, 10));

interactive(children=(IntSlider(value=5, description='n_rows', max=10, min=1), IntSlider(value=5, description=…